In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils.metric_utils import HardMetric
from sklearn.metrics import f1_score

%matplotlib inline

In [35]:
VALIDATION = False
SAMPLE = True
SAMPLE_FRAC = 0.3
final_metric = HardMetric(f1_score)

In [36]:
train = pd.read_csv('data/train.csv',index_col = 'id')
test = pd.read_csv('data/test.csv',index_col = 'id')
data = pd.concat([train, test]).drop(columns = 'target')
sample_test = pd.read_csv('data/sample_submission.csv',index_col = 'id')

C:\Alon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
from sklearn.model_selection import train_test_split
if VALIDATION:
    if SAMPLE:
        data = train.sample(frac = SAMPLE_FRAC)
    else:
        data = train
    train, test = train_test_split(data, train_size=0.7, shuffle=True)
    y_test = test['target']
    test = test.drop(columns = 'target')
else:
    train = train.sample(frac = 1.)

In [38]:
train.shape, test.shape

((7613, 4), (3263, 3))

# Feature Extraction

## Keyword

In [39]:
from feature_extraction.keyword import process_keyword

keyword = data['keyword']
X_keyword = process_keyword(keyword)
X_keyword.sample(10)

,keyword_OH_ablaze,keyword_OH_accident,keyword_OH_aftershock,keyword_OH_airplane%20accident,keyword_OH_ambulance,keyword_OH_annihilated,keyword_OH_annihilation,keyword_OH_apocalypse,keyword_OH_armageddon,keyword_OH_army,...,keyword_OH_whirlwind,keyword_OH_wild%20fires,keyword_OH_wildfire,keyword_OH_windstorm,keyword_OH_wounded,keyword_OH_wounds,keyword_OH_wreck,keyword_OH_wreckage,keyword_OH_wrecked,keyword_isna
id,,,,,,,,,,,,,,,,,,,,,
2253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Text

In [40]:
from feature_extraction.text import process_text

text = data['text']
X_text = process_text(text)
X_text.sample(10)

,text_our,text_deeds,text_are,text_the,text_reason,text_of,text_this,text_earthquake,text_may,text_allah,...,text_reserve,text_hattrick,text_ebolaoutbreak,text_ala,text_issuicide,text_rajman,text_hasaka,text_risen,text_fasteners,text_xrwn
id,,,,,,,,,,,,,,,,,,,,,
8842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## combine

In [41]:
X_all = X_keyword.join(X_text)

X_train = X_all.loc[train.index]
y_train = train['target']
X_test = X_all.loc[test.index]

# Modeling

In [42]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

## Fitting

In [ ]:
clf.fit(X_train, y_train, )
y_pred_score = clf.predict_proba(X_test)[:, 1]

## Threshold choosing

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred_score = cross_val_predict(clf, X_train, y_train, method = 'predict_proba')[:, 1]
th = final_metric.get_thresh(y_train, y_train_pred_score)

In [ ]:
y_pred = (y_pred_score > th).astype(int)

# (Validation) Evaluation

In [ ]:
if VALIDATION:
    from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, plot_roc_curve, plot_confusion_matrix, plot_precision_recall_curve
    import matplotlib.pyplot as plt
    print(f'total score:\t {final_metric.metric(y_test, y_pred)}')
    plot_roc_curve(clf, X_test, y_test)
    plot_precision_recall_curve(clf, X_test, y_test)
    plot_confusion_matrix(clf, X_test, y_test)

# (Pred) Save Results

In [ ]:
if not VALIDATION:
    import datetime
    
    current_time = str(datetime.datetime.now())[:-7].replace(' ', '__').replace('-', '_').replace(':', '_')
    pd.Series(data = y_pred, index = test.index, name='target').to_frame().to_csv(f'results/results_{current_time}.csv')

In [ ]:
y_pred.shape

In [ ]:
y_pred.sum()